# **Part 1: Setting up `Cart-Pole` Agent.**

- `Name` : **Pavaris Asawakijtananont**
- `Number` : 65340500037

## **1. RL Base class**

### **Constructor** 

### **Core Function**

From the Cart-Pole environment configuration file, we found that the observation space of the Cart-Pole consists of four components:

1. Relative joint velocity of the cart.
2. Relative joint velocity of the pole.
3. Relative joint position of the cart.
4. Relative joint position of the pole.

Each component is continuous and has an $\infty$ range, which results in high computational costs and an excessively large number of states. To discretize the observation state, we must first determine the complete range of values in the environment. Some of these parameters can be obtained from the Cart-Pole configuration file.

```python
CARTPOLE_CFG = ArticulationCfg(
    spawn=sim_utils.UsdFileCfg(
        usd_path=f"{ISAACLAB_NUCLEUS_DIR}/Robots/Classic/Cartpole/cartpole.usd",
        rigid_props=sim_utils.RigidBodyPropertiesCfg(
            rigid_body_enabled=True,
            max_linear_velocity=1000.0,
            max_angular_velocity=1000.0,
            max_depenetration_velocity=100.0,
            enable_gyroscopic_forces=True,
        ),
        articulation_props=sim_utils.ArticulationRootPropertiesCfg(
            enabled_self_collisions=False,
            solver_position_iteration_count=4,
            solver_velocity_iteration_count=0,
            sleep_threshold=0.005,
            stabilization_threshold=0.001,
        ),
    ),
    init_state=ArticulationCfg.InitialStateCfg(
        pos=(0.0, 0.0, 2.0), joint_pos={"slider_to_cart": 0.0, "cart_to_pole": 0.0}
    ),
    actuators={
        "cart_actuator": ImplicitActuatorCfg(
            joint_names_expr=["slider_to_cart"],
            effort_limit=400.0,
            velocity_limit=100.0,
            stiffness=0.0,
            damping=10.0,
        ),
        "pole_actuator": ImplicitActuatorCfg(
            joint_names_expr=["cart_to_pole"], effort_limit=400.0, velocity_limit=100.0, stiffness=0.0, damping=0.0
        ),
    },
)
```

1. `pose_cart` : cart position we can set up on termination condition and we must have tolerance before terminate
2. `pose_pol` : Isaac Sim have angle between **[ -$\pi$ , $\pi$ ]**
3. `vel_cart` : velocity of cart is determine by `velocity_limit` of joint `slider_to_cart` is **[-100 , 100]**
4. `vel_pol` : actuator set in **100 rad/s** 